### unbalancing solution

### gini coefficient

https://www.kaggle.com/batzner/gini-coefficient-an-intuitive-explanation

### preprocessing

filling

dummy

normalization

difficulty on preprocessing after cv

In [17]:
import warnings 
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import xgboost as xgb

from preprocessing import dataset
from sklearn import metrics

In [18]:
def loadcv(path):
    cv = []
    i = 0
    while True:
        try:
            train = dataset(pd.read_csv(path+"train_X_"+str(i)+".csv"),
                    pd.read_csv(path+"train_y_"+str(i)+".csv"))
            valid = dataset(pd.read_csv(path+"valid_X_"+str(i)+".csv"),
                    pd.read_csv(path+"valid_y_"+str(i)+".csv"))
            cv.append([train, valid])
            print("cross validation set %d loaded"%i)
            i += 1
        except:
            return cv

In [19]:
path = "./cv/cv_"
cv = loadcv(path)

cross validation set 0 loaded
cross validation set 1 loaded
cross validation set 2 loaded
cross validation set 3 loaded
cross validation set 4 loaded


In [20]:
test = dataset(pd.read_csv(path+"test_X"+".csv"),
               pd.read_csv(path+"test_y"+".csv"))

In [29]:
def gini(y, pred):
    fpr, tpr, thr = metrics.roc_curve(y, pred, pos_label=1)
    g = 2 * metrics.auc(fpr, tpr) -1
    return g

def gini_xgb(pred, y):
    y = y.get_label()
    return 'gini', gini(y, pred)

In [31]:
params = {'eta': 0.02, 
          'max_depth': 4, 
          'subsample': 0.9, 
          'colsample_bytree': 0.9, 
          'objective': 'binary:logistic', 
          'eval_metric': 'auc', 
          'seed': 99, 
          'silent': True
         }
result = []
for i,data in enumerate(cv):
    print("training CV round %d"%i)
    watchlist = [(xgb.DMatrix(data[0].X, data[0].y), 'train'), (xgb.DMatrix(data[1].X, data[1].y), 'valid')]
    model = xgb.train(params, xgb.DMatrix(data[0].X, data[0].y), 5000,  watchlist, feval=gini_xgb, maximize=True, verbose_eval=50, early_stopping_rounds=200)
    result.append( model.predict(xgb.DMatrix(test.X), ntree_limit=model.best_ntree_limit+45) )

training CV round 0
[0]	train-gini:0.201878	valid-gini:0.192991
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 200 rounds.
[50]	train-gini:0.245915	valid-gini:0.225745
[100]	train-gini:0.250254	valid-gini:0.230779
[150]	train-gini:0.262452	valid-gini:0.238926
[200]	train-gini:0.277772	valid-gini:0.249345
[250]	train-gini:0.290113	valid-gini:0.257182
[300]	train-gini:0.299726	valid-gini:0.262498
[350]	train-gini:0.307902	valid-gini:0.265866
[400]	train-gini:0.313991	valid-gini:0.267796
[450]	train-gini:0.320037	valid-gini:0.26946
[500]	train-gini:0.324973	valid-gini:0.270824
[550]	train-gini:0.330432	valid-gini:0.27155
[600]	train-gini:0.33535	valid-gini:0.272167
[650]	train-gini:0.339975	valid-gini:0.272679
[700]	train-gini:0.344483	valid-gini:0.272751
[750]	train-gini:0.348959	valid-gini:0.272984
[800]	train-gini:0.353313	valid-gini:0.273409
[850]	train-gini:0.357613	valid-gini:0.273119
[900]	train-

ValueError: feature_names mismatch: [u'ps_ind_01', u'ps_ind_03', u'ps_ind_06_bin', u'ps_ind_07_bin', u'ps_ind_08_bin', u'ps_ind_09_bin', u'ps_ind_10_bin', u'ps_ind_11_bin', u'ps_ind_12_bin', u'ps_ind_13_bin', u'ps_ind_14', u'ps_ind_15', u'ps_ind_16_bin', u'ps_ind_17_bin', u'ps_ind_18_bin', u'ps_reg_01', u'ps_reg_02', u'ps_reg_03', u'ps_car_01_cat', u'ps_car_04_cat', u'ps_car_06_cat', u'ps_car_11_cat', u'ps_car_11', u'ps_car_12', u'ps_car_13', u'ps_car_14', u'ps_car_15', u'ps_calc_01', u'ps_calc_02', u'ps_calc_03', u'ps_calc_04', u'ps_calc_05', u'ps_calc_06', u'ps_calc_07', u'ps_calc_08', u'ps_calc_09', u'ps_calc_10', u'ps_calc_11', u'ps_calc_12', u'ps_calc_13', u'ps_calc_14', u'ps_calc_15_bin', u'ps_calc_16_bin', u'ps_calc_17_bin', u'ps_calc_18_bin', u'ps_calc_19_bin', u'ps_calc_20_bin', u'1hot_ps_ind_02_-1.0', u'1hot_ps_ind_02_1.0', u'1hot_ps_ind_02_2.0', u'1hot_ps_ind_02_3.0', u'1hot_ps_ind_02_4.0', u'1hot_ps_ind_04_-1.0', u'1hot_ps_ind_04_0.0', u'1hot_ps_ind_04_1.0', u'1hot_ps_ind_0

In [38]:
test.y['target'] = np.mean(result,axis = 0)

In [40]:
test.y.to_csv('xgb_submission.csv', index=False, float_format='%.5f')

### Dependencies

In [4]:
import warnings 
warnings.filterwarnings("ignore")
import pandas as pd
import numpy  as np
# from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
# import xgboost as xgb
from multiprocessing import cpu_count

In [5]:
from collections import namedtuple
dataset = namedtuple("dataset","X y")

### Loading Raw Data

In [27]:
train = pd.read_csv("./original/train.csv")
test  = pd.read_csv("./original/test.csv")

In [28]:
class dataset:
    def __init__(self, data = None, label = None):
        self.X = data
        self.y = label

### Preprocessing

In [29]:
train = dataset(train.drop("id",1).drop("target",1), train.target)
test  = dataset(test.drop("id",1), test.id)

In [30]:
train.X = train.X.replace(-1, np.nan)
test.X  = test .X.replace(-1, np.nan)

##### fillNA

In [31]:
train.X = train.X.fillna(-1)
test.X = test.X.fillna(-1)

##### Dummy Catagories

In [26]:
#Although it is better to do everything after splitting train-validation
#we extract dummylist first in order not to have different dimension for different sets
CATAGORY_THRES = 10
def dummylist(df):
    assert(isinstance(df,pd.core.frame.DataFrame))
    dummylist = []
    for columnname in list(df.columns):
        if columnname.endswith("cat") and len(list(df[columnname].unique())) < CATAGORY_THRES:
            dummylist.append(columnname)
    return dummylist
    return df    
def dummy(df, dummylist):
    assert(isinstance(df,pd.core.frame.DataFrame))
    for columnname in dummylist:
        newcolumn = pd.get_dummies(df[columnname],prefix = "1hot_"+columnname[:-4])
        df = pd.concat([df, newcolumn],axis = 1)
    for columnname in dummylist:
        df = df.drop(columnname, axis = 1)
    return df

In [27]:
mylist = dummylist(train.X)

In [28]:
train.X = dummy(train.X, mylist)
test.X  = dummy(test.X,  mylist)

In [29]:
KFOLDS = 5
folds = list(StratifiedKFold(n_splits=KFOLDS, shuffle=True, random_state=2016).split(train.X, train.y))

In [32]:
tmp = pd.DataFrame.copy(train.X.head(10))

In [33]:
tmp['ps_ind_01'][0] = 3

In [34]:
train.X.head()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,2,2.0,5,1.0,0.0,0,1,0,0,0,...,9,1,5,8,0,1,1,0,0,1
1,1,1.0,7,0.0,0.0,0,0,1,0,0,...,3,1,1,9,0,1,1,0,1,0
2,5,4.0,9,1.0,0.0,0,0,1,0,0,...,4,2,7,7,0,1,1,0,1,0
3,0,1.0,2,0.0,0.0,1,0,0,0,0,...,2,2,4,9,0,0,0,0,0,0
4,0,2.0,0,1.0,0.0,1,0,0,0,0,...,3,1,1,3,0,0,0,1,1,0


In [35]:
tmp

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,3,2.0,5,1.0,0.0,0,1,0,0,0,...,9,1,5,8,0,1,1,0,0,1
1,1,1.0,7,0.0,0.0,0,0,1,0,0,...,3,1,1,9,0,1,1,0,1,0
2,5,4.0,9,1.0,0.0,0,0,1,0,0,...,4,2,7,7,0,1,1,0,1,0
3,0,1.0,2,0.0,0.0,1,0,0,0,0,...,2,2,4,9,0,0,0,0,0,0
4,0,2.0,0,1.0,0.0,1,0,0,0,0,...,3,1,1,3,0,0,0,1,1,0
5,5,1.0,4,0.0,0.0,0,0,0,1,0,...,4,2,0,9,0,1,0,1,1,1
6,2,1.0,3,1.0,0.0,0,1,0,0,0,...,3,0,0,10,0,1,0,0,1,0
7,5,1.0,4,0.0,0.0,1,0,0,0,0,...,7,1,3,6,1,0,1,0,1,0
8,5,1.0,3,1.0,0.0,0,0,1,0,0,...,4,2,1,5,0,1,0,0,0,1
9,1,1.0,2,0.0,0.0,0,1,0,0,0,...,3,5,0,6,0,1,0,0,1,0


In [36]:
df = pd.DataFrame(np.arange(36).reshape(9,4),
                      columns=['A', 'B', 'C', 'D'])

In [42]:
pd.concat([df.drop(range(4)),df.drop(range(4))],axis = 0).reset_index()

,index,A,B,C,D
0,4,16,17,18,19
1,5,20,21,22,23
2,6,24,25,26,27
3,7,28,29,30,31
4,8,32,33,34,35
5,4,16,17,18,19
6,5,20,21,22,23
7,6,24,25,26,27
8,7,28,29,30,31
9,8,32,33,34,35
